# Análisis estadístico para la banda @490 utilizando una máscara

Primero importaremos las librerías necesarias.

In [1]:
import numpy as np
import pandas

Leemos los datos para enero.

In [2]:
# Para el sensor SeaWiFS.

with open('./ENERO_2003/SeaWiFS/Composicion (Lt_490).txt') as archivo_seawifs:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_seawifs.readline()
    # Separamos los datos en una matriz.
    datos_seawifs = np.array([archivo_seawifs.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_seawifs = np.array([[float(dat) for dat in row] for row in datos_seawifs])

In [3]:
# Para el sensor MODIS. Es lo mismo pero con otro archivo.

with open('./ENERO_2003/MODIS/Composicion (Band 10).txt') as archivo_modis:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_modis.readline()
    # Separamos los datos en una matriz.
    datos_modis = np.array([archivo_modis.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_modis = np.array([[float(dat) for dat in row] for row in datos_modis])

Revisamos datos estadísticos de los datos sucios.

In [4]:
media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS: {}.".format(var_modis))

Promedio de los datos del SeaWiFS: 44.80587750011788.
Promedio de los datos del MODIS: 42.81177717478429.
Varianza de los datos del SeaWiFS: 34.116755167250844.
Varianza de los datos del MODIS: 15.516441651737484.


### Función de la máscara

In [5]:
def filter(img, row, column, tol):
    mean_img = np.mean(img)
    
    for i in range(row):
        row_sum = 0.0
        row_mean = 0.0
        
        for j in range(column):
            row_sum += img[i][j]
            row_mean = row_sum / (j + 1)
            
            if np.abs(img[i][j] - mean_img) >= tol:
                #img[i][j] = row_mean
                img[i][j] = mean_img

In [5]:
def get_neighbors(img, i, j, row, column):
    neigh = []
    
    for k in range(i-1, i+2):
        for l in range(j-1, j+2):
            if k == i and l == j:
                continue
            elif k < 0 or l < 0 or k >= row or l >= column:
                #neigh.append(0)
                continue
            else:
                neigh.append(img[k][l])
    
    return neigh

In [6]:
def filter(img, row, column, tol):
    mean_img = np.mean(img)
    
    for i in range(row):
        for j in range(column):
            if np.abs(img[i][j] - mean_img) >= tol:
                img[i][j] = np.mean(get_neighbors(img, i, j, row, column))

Ahora limpiamos las composiciones.

In [6]:
filter(datos_seawifs, 127, 167, 3)
datos_seawifs

array([[43.2616   , 43.211    , 43.211    , ..., 44.8058775, 44.8058775,
        44.8058775],
       [43.3355   , 43.2849   , 43.2815   , ..., 44.8058775, 44.8058775,
        44.8058775],
       [46.1365   , 45.978    , 45.9745   , ..., 44.8058775, 44.8058775,
        44.8058775],
       ...,
       [44.8058775, 44.8058775, 44.8058775, ..., 43.7049   , 43.7119   ,
        43.6942   ],
       [44.8058775, 44.8058775, 44.8058775, ..., 43.6696   , 43.6732   ,
        43.6555   ],
       [44.8058775, 44.8058775, 44.8058775, ..., 43.6696   , 43.6732   ,
        43.6555   ]])

In [7]:
filter(datos_modis, 127, 167, 3)
datos_modis

array([[41.1685    , 41.1685    , 41.1422    , ..., 42.81177717,
        42.81177717, 42.81177717],
       [41.1632    , 41.1632    , 41.1404    , ..., 42.81177717,
        42.81177717, 42.81177717],
       [41.1659    , 41.1659    , 41.1199    , ..., 42.81177717,
        42.81177717, 42.81177717],
       ...,
       [42.81177717, 42.81177717, 42.81177717, ..., 42.5495    ,
        42.5752    , 42.5772    ],
       [42.81177717, 42.81177717, 42.81177717, ..., 42.5521    ,
        42.5816    , 42.5484    ],
       [42.81177717, 42.81177717, 42.81177717, ..., 42.5366    ,
        42.5614    , 42.5545    ]])

In [8]:
media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS: {}.".format(var_modis))

Promedio de los datos del SeaWiFS: 43.384122612815545.
Promedio de los datos del MODIS: 41.846795899371735.
Varianza de los datos del SeaWiFS: 0.6063902883176688.
Varianza de los datos del MODIS: 0.4511020233714358.


In [9]:
datos_comb = abs(datos_seawifs - datos_modis)
ma_comb = np.mean(datos_comb)
var_comb = np.var(datos_comb)

print("Media aritmética de las distancias entre pixeles: {}".format(ma_comb))
print("Varianza de las distancias entre pixeles: {}.".format(var_comb))

Media aritmética de las distancias entre pixeles: 1.5407423174324502
Varianza de las distancias entre pixeles: 0.288495615701545.


## Febrero

In [10]:
# Para el sensor SeaWiFS.

with open('./FEBRERO_2003/SeaWiFS/3_Bandas/Composicion (Lt_490).txt') as archivo_seawifs:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_seawifs.readline()
    # Separamos los datos en una matriz.
    datos_seawifs = np.array([archivo_seawifs.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_seawifs = np.array([[float(dat) for dat in row] for row in datos_seawifs])

# Para el sensor MODIS. Es lo mismo pero con otro archivo.

with open('./FEBRERO_2003/MODIS/3_Bandas/Composicion (B10).txt') as archivo_modis:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_modis.readline()
    # Separamos los datos en una matriz.
    datos_modis = np.array([archivo_modis.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_modis = np.array([[float(dat) for dat in row] for row in datos_modis])

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS sin limpiar: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS sin limpiar: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS sin limpiar: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS sin limpiar: {}.".format(var_modis))

##-----------------------------------------------

filter(datos_seawifs, 127, 167, 3)
filter(datos_modis, 127, 167, 3)

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS limpios: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS limpios: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS limpios: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS limpios: {}.".format(var_modis))

##-----------------------------------------------

datos_comb = abs(datos_seawifs - datos_modis)
ma_comb = np.mean(datos_comb)
var_comb = np.var(datos_comb)

print("Media aritmética de las distancias entre pixeles: {}".format(ma_comb))
print("Varianza de las distancias entre pixeles: {}.".format(var_comb))

Promedio de los datos del SeaWiFS sin limpiar: 50.37943367909849.
Promedio de los datos del MODIS sin limpiar: 43.087711377245505.
Varianza de los datos del SeaWiFS sin limpiar: 107.25804162019581.
Varianza de los datos del MODIS sin limpiar: 20.39668228312531.
Promedio de los datos del SeaWiFS limpios: 50.1973334481781.
Promedio de los datos del MODIS limpios: 41.929372223519394.
Varianza de los datos del SeaWiFS limpios: 0.6207501230113712.
Varianza de los datos del MODIS limpios: 0.6627445715691115.
Media aritmética de las distancias entre pixeles: 8.267961224658707
Varianza de las distancias entre pixeles: 1.337468400437053.


## Marzo

In [11]:
# Para el sensor SeaWiFS.

with open('./MARZO_2003/SeaWiFS/Composicion (Lt_490).txt') as archivo_seawifs:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_seawifs.readline()
    # Separamos los datos en una matriz.
    datos_seawifs = np.array([archivo_seawifs.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_seawifs = np.array([[float(dat) for dat in row] for row in datos_seawifs])

# Para el sensor MODIS. Es lo mismo pero con otro archivo.

with open('./MARZO_2003/MODIS/Composicion (B10).txt') as archivo_modis:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_modis.readline()
    # Separamos los datos en una matriz.
    datos_modis = np.array([archivo_modis.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_modis = np.array([[float(dat) for dat in row] for row in datos_modis])

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS sin limpiar: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS sin limpiar: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS sin limpiar: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS sin limpiar: {}.".format(var_modis))

##-----------------------------------------------

filter(datos_seawifs, 127, 167, 3)
filter(datos_modis, 127, 167, 3)

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS limpios: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS limpios: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS limpios: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS limpios: {}.".format(var_modis))

##-----------------------------------------------

datos_comb = abs(datos_seawifs - datos_modis)
ma_comb = np.mean(datos_comb)
var_comb = np.var(datos_comb)

print("Media aritmética de las distancias entre pixeles: {}".format(ma_comb))
print("Varianza de las distancias entre pixeles: {}.".format(var_comb))

Promedio de los datos del SeaWiFS sin limpiar: 53.67939175821586.
Promedio de los datos del MODIS sin limpiar: 52.79657578858032.
Varianza de los datos del SeaWiFS sin limpiar: 103.00748182330138.
Varianza de los datos del MODIS sin limpiar: 22.182261898164807.
Promedio de los datos del SeaWiFS limpios: 53.541051998247056.
Promedio de los datos del MODIS limpios: 51.784627739957244.
Varianza de los datos del SeaWiFS limpios: 0.49609382728836876.
Varianza de los datos del MODIS limpios: 0.5004560786188818.
Media aritmética de las distancias entre pixeles: 1.8826501593877198
Varianza de las distancias entre pixeles: 0.490253116738869.


## Abril

In [12]:
# Para el sensor SeaWiFS.

with open('./ABRIL_2003_CORREGIDOS/SeaWiFS/Composicion (B490).txt') as archivo_seawifs:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_seawifs.readline()
    # Separamos los datos en una matriz.
    datos_seawifs = np.array([archivo_seawifs.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_seawifs = np.array([[float(dat) for dat in row] for row in datos_seawifs])

# Para el sensor MODIS. Es lo mismo pero con otro archivo.

with open('./ABRIL_2003_CORREGIDOS/MODIS/Composicion (B10).txt') as archivo_modis:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_modis.readline()
    # Separamos los datos en una matriz.
    datos_modis = np.array([archivo_modis.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_modis = np.array([[float(dat) for dat in row] for row in datos_modis])

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS sin limpiar: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS sin limpiar: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS sin limpiar: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS sin limpiar: {}.".format(var_modis))

##-----------------------------------------------

filter(datos_seawifs, 127, 167, 3)
filter(datos_modis, 127, 167, 3)

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS limpios: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS limpios: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS limpios: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS limpios: {}.".format(var_modis))

##-----------------------------------------------

datos_comb = abs(datos_seawifs - datos_modis)
ma_comb = np.mean(datos_comb)
var_comb = np.var(datos_comb)

print("Media aritmética de las distancias entre pixeles: {}".format(ma_comb))
print("Varianza de las distancias entre pixeles: {}.".format(var_comb))

Promedio de los datos del SeaWiFS sin limpiar: 54.72241087745769.
Promedio de los datos del MODIS sin limpiar: 60.11794838983451.
Varianza de los datos del SeaWiFS sin limpiar: 60.94909096467634.
Varianza de los datos del MODIS sin limpiar: 40.49489379521465.
Promedio de los datos del SeaWiFS limpios: 53.0054125497588.
Promedio de los datos del MODIS limpios: 58.65376410194865.
Varianza de los datos del SeaWiFS limpios: 0.9352028398023592.
Varianza de los datos del MODIS limpios: 0.7747125567347789.
Media aritmética de las distancias entre pixeles: 5.648351552189855
Varianza de las distancias entre pixeles: 0.5554852746263171.


## Mayo

In [13]:
# Para el sensor SeaWiFS.

with open('./MAYO_2003/SeaWiFS/Composicion (Lt_490).txt') as archivo_seawifs:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_seawifs.readline()
    # Separamos los datos en una matriz.
    datos_seawifs = np.array([archivo_seawifs.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_seawifs = np.array([[float(dat) for dat in row] for row in datos_seawifs])

# Para el sensor MODIS. Es lo mismo pero con otro archivo.

with open('./MAYO_2003/MODIS/Composicion (B10).txt') as archivo_modis:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_modis.readline()
    # Separamos los datos en una matriz.
    datos_modis = np.array([archivo_modis.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_modis = np.array([[float(dat) for dat in row] for row in datos_modis])

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS sin limpiar: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS sin limpiar: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS sin limpiar: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS sin limpiar: {}.".format(var_modis))

##-----------------------------------------------

filter(datos_seawifs, 127, 167, 3)
filter(datos_modis, 127, 167, 3)

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS limpios: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS limpios: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS limpios: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS limpios: {}.".format(var_modis))

##-----------------------------------------------

datos_comb = abs(datos_seawifs - datos_modis)
ma_comb = np.mean(datos_comb)
var_comb = np.var(datos_comb)

print("Media aritmética de las distancias entre pixeles: {}".format(ma_comb))
print("Varianza de las distancias entre pixeles: {}.".format(var_comb))

Promedio de los datos del SeaWiFS sin limpiar: 56.658503922862934.
Promedio de los datos del MODIS sin limpiar: 70.36671088217265.
Varianza de los datos del SeaWiFS sin limpiar: 59.011802082315704.
Varianza de los datos del MODIS sin limpiar: 15.154067451401215.
Promedio de los datos del SeaWiFS limpios: 54.88979369714663.
Promedio de los datos del MODIS limpios: 69.47821361892314.
Varianza de los datos del SeaWiFS limpios: 0.9300941840726578.
Varianza de los datos del MODIS limpios: 1.3737651675935334.
Media aritmética de las distancias entre pixeles: 14.588419921776515
Varianza de las distancias entre pixeles: 1.7085808849753719.


## Junio

In [14]:
# Para el sensor SeaWiFS.

with open('./JUNIO_2003/SeaWiFS/Composicion (Lt_490).txt') as archivo_seawifs:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_seawifs.readline()
    # Separamos los datos en una matriz.
    datos_seawifs = np.array([archivo_seawifs.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_seawifs = np.array([[float(dat) for dat in row] for row in datos_seawifs])

# Para el sensor MODIS. Es lo mismo pero con otro archivo.

with open('./JUNIO_2003/MODIS/Composicion (B10).txt') as archivo_modis:
    for _ in range(5):
        # Ignoramos los primeros 5 renglones del archivo.
        archivo_modis.readline()
    # Separamos los datos en una matriz.
    datos_modis = np.array([archivo_modis.readline().split()
                              for _ in range(127)])

# Convertimos a reales.
datos_modis = np.array([[float(dat) for dat in row] for row in datos_modis])

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS sin limpiar: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS sin limpiar: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS sin limpiar: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS sin limpiar: {}.".format(var_modis))

##-----------------------------------------------

filter(datos_seawifs, 127, 167, 3)
filter(datos_modis, 127, 167, 3)

##-----------------------------------------------

media_seawifs = np.mean(datos_seawifs)
media_modis = np.mean(datos_modis)

var_seawifs = np.var(datos_seawifs)
var_modis = np.var(datos_modis)

print("Promedio de los datos del SeaWiFS limpios: {}.".format(media_seawifs))
print("Promedio de los datos del MODIS limpios: {}.".format(media_modis))

print("Varianza de los datos del SeaWiFS limpios: {}.".format(var_seawifs))
print("Varianza de los datos del MODIS limpios: {}.".format(var_modis))

##-----------------------------------------------

datos_comb = abs(datos_seawifs - datos_modis)
ma_comb = np.mean(datos_comb)
var_comb = np.var(datos_comb)

print("Media aritmética de las distancias entre pixeles: {}".format(ma_comb))
print("Varianza de las distancias entre pixeles: {}.".format(var_comb))

Promedio de los datos del SeaWiFS sin limpiar: 56.22971067471356.
Promedio de los datos del MODIS sin limpiar: 63.00103334433496.
Varianza de los datos del SeaWiFS sin limpiar: 54.710943638967585.
Varianza de los datos del MODIS sin limpiar: 29.559309876998388.
Promedio de los datos del SeaWiFS limpios: 55.224210473960454.
Promedio de los datos del MODIS limpios: 62.77184685285412.
Varianza de los datos del SeaWiFS limpios: 1.4417400180700153.
Varianza de los datos del MODIS limpios: 1.626686294292506.
Media aritmética de las distancias entre pixeles: 7.547636378893666
Varianza de las distancias entre pixeles: 3.001205534837544.
